In [1]:
import cv2 as cv
import numpy as np
import csv
import os

# Ścieżki plików
csv_input = 'coins.csv'  # CSV z listą nazw plików do przetworzenia
csv_output = 'wyniki_2.csv'       # CSV z wynikami (nazwa pliku, koordynaty, ścieżka do wycinka)
output_dir = 'cropped3'          # Katalog na wycinki

# Upewniamy się, że katalog istnieje
os.makedirs(output_dir, exist_ok=True)

with open(csv_output, 'w', newline='', encoding='utf-8') as f_out:
    writer = csv.writer(f_out, delimiter=';')
    # Nagłówek w pliku wynikowym
    writer.writerow(['img_dir', 'label', 'x1', 'y1', 'x2', 'y2', 'cropped_path'])
    
    with open(csv_input, 'r', encoding='utf-8') as f_in:
        reader = csv.reader(f_in, delimiter=";")
        next(reader)
        for row in reader:
            if not row:
                continue
            input_file = "./Monety/" + row[1]

            # Wczytujemy obraz w odcieniach szarości
            image = cv.imread(input_file)
            if image is None:
                print(f"Nie można wczytać obrazu: {input_file}")
                writer.writerow([row[1], row[0], None, None, None, None, "blad_wczytania"])
                continue
            
            output = image.copy()
            
            gray = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
            hist = cv.equalizeHist(gray)
            blur = cv.GaussianBlur(hist, (31,31), cv.BORDER_DEFAULT)
            
            height, width = blur.shape[:2]

            minR = round(width/25)
            maxR = round(width/5)
            minDis = round(width/3)
            
            circles = cv.HoughCircles(blur,cv.HOUGH_GRADIENT,1,minDis,
                            param1=100,param2=30,minRadius=0,maxRadius=maxR)
            
            
            image_blur = cv.medianBlur(image, 5)

            # Wykrywanie okręgów
            circles = cv.HoughCircles(
                image_blur,
                cv.HOUGH_GRADIENT,
                dp=2,
                minDist=20,
                param1=150,
                param2=100,
                minRadius=500,
                maxRadius=2500
            )

            if circles is not None:
                circles = np.uint16(np.around(circles))
                # Największy okrąg
                largest_circle = max(circles[0, :], key=lambda x: x[2])
                
                # Konwersja na int aby uniknąć problemów z typem uint16
                x, y, r = map(int, largest_circle)

                # Koordynaty wycinka
                x1, y1 = x - r, y - r
                x2, y2 = x + r, y + r

                # Upewniamy się, że koordynaty mieszczą się w granicach obrazu
                # h, w = blur.shape[:2]
                x1 = max(0, x1)
                y1 = max(0, y1)
                x2 = min(width, x2)
                y2 = min(height, y2)

                # Sprawdzamy czy wycinek jest niepusty
                if x2 <= x1 or y2 <= y1:
                    print(f"Nieprawidłowy wycinek dla obrazu: {input_file}")
                    writer.writerow([row[1], row[0], x1, y1, x2, y2, "nieprawidlowy_wycinek"])
                    continue

                cropped_circle = output[y1:y2, x1:x2]

                if cropped_circle.size == 0:
                    # Pusty obraz - nie zapisujemy
                    writer.writerow([row[1], row[0], x1, y1, x2, y2, "pusty_wycinek"])
                    continue

                # Zapisujemy wycinek
                base_name = os.path.basename(input_file)
                output_path = os.path.join(output_dir, f"crop_{base_name}")
                success = cv.imwrite(output_path, cropped_circle)
                if success:
                    writer.writerow([row[1], row[0], x1, y1, x2, y2, output_path])
                else:
                    writer.writerow([row[1], row[0], x1, y1, x2, y2, "blad_zapisu"])
            else:
                # Brak wykrytego okręgu
                writer.writerow([row[1], row[0], None, None, None, None, "brak_okręgu"])